<a href="https://colab.research.google.com/github/SeungHan0816/Colab_Practice/blob/main/Colab_Practice07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07-01 순환 신경망(Recurrent Neural Network, RNN)

In [ ]:
import numpy as np

# 은닉 상태 : 이전까지 입력된 정보들을 요약하여 다음 계산에 전달하기 위한 중간 기억 벡터

# (timesteps, input_size) 크기의 2D 텐서를 입력을 받음.
# 실제로는 (batch_size, timesteps, input_size)의 크기의 3D 텐서를 입력을 받음.

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_size = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다.

# np.random은 랜덤 관련 기능 모아둔 "모듈"
# np.random.random(...)은 그 모듈 안에 있는 random() 함수

inputs = np.random.random((timesteps, input_size)) # 입력에 해당되는 2D 텐서

hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬.

print(hidden_state_t) # 8의 크기를 가지는 은닉 상태. 현재는 초기 은닉 상태로 모든 차원이 0의 값을 가짐.

print("-"*100)

Wx = np.random.random((hidden_size, input_size))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_size,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias)

print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

print("-"*100)

total_hidden_states = []

# 메모리 셀 동작
# np.dot은 행렬 곱
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) # Wx * Xt + Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적

  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t

# 리스트에 쌓아둔 은닉 상태들을 하나의 배열로 합쳐서 차원 올리기
# 새 축(axis=0)을 기준으로 쌓는다.
total_hidden_states = np.stack(total_hidden_states, axis = 0)
# 출력 시 값을 깔끔하게 해준다.

print("-"*100)

# (timesteps, output_dim)의 크기. 이 경우 (10, 8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력.
print(total_hidden_states)

print("-"*100)



[0. 0. 0. 0. 0. 0. 0. 0.]
----------------------------------------------------------------------------------------------------
(8, 4)
(8, 8)
(8,)
----------------------------------------------------------------------------------------------------
(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
----------------------------------------------------------------------------------------------------
[[0.91990077 0.96246543 0.86038489 0.78653153 0.92924253 0.98944964
  0.80188456 0.94470339]
 [0.99980968 0.99997092 0.9998881  0.99741157 0.99999095 0.99999464
  0.99984992 0.99983684]
 [0.99988954 0.99995005 0.99985266 0.99752138 0.99999339 0.99999248
  0.99984885 0.99990442]
 [0.9999666  0.99998776 0.99997285 0.99913669 0.99999719 0.99999803
  0.99996182 0.99992761]
 [0.99995897 0.99997492 0.99993764 0.99884728 0.99999602 0.9999975
  0.99994506 0.99994944]
 [0.99991096 0.9999748  0.99989082 0.99825824 0.999994   0.9999957
  0.99989857 0.99992675]
 [0.99983582 0.99992107 0

In [ ]:
import torch
import torch.nn as nn

# 은닉 상태는 내부 계산 결과고, 그걸 바깥으로 내보내면 출력이 되는 거니까,
# 출력과 은닉 상태를 같다고 말하는 경우가 있다

input_size = 5 # 입력의 크기
hidden_size = 8 # 은닉 상태의 크기

# 입력 텐서는 (배치 크기 × 시점의 수 × 매 시점마다 들어가는 입력)의 크기
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

# RNN의 셀 인자로 입력의 크기, 은닉 상태의 크기를 정의해주고,
# batch_first=True를 통해서 입력 텐서의 첫번째 차원이 배치 크기임을 알려줌
cell = nn.RNN(input_size, hidden_size, batch_first=True)

# _something	덜 중요하거나, 부가적인 용도로 쓰일 변수
# outputs에 집중하고, status는 부가적인 정보

# outputs는 여러 개의 은닉 상태(h₁, h₂, ..., hₜ)고,
# status (정확히는 h_n)는 마지막 은닉 상태를 자동으로 반환해주는 RNN의 출력값 중 하나

outputs, _status = cell(inputs)

print(outputs.shape) # 모든 time-step의 hidden_state

# 마지막 시점의 은닉 상태만 딱 하나 추출, 1개 배치, 1개 시점(마지막), 8차원 은닉 상태
print(_status.shape) # 최종 time-step의 hidden_state

print("-"*100)

# 깊은 순환 신경망(Deep Recurrent Neural Network)

# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True)
outputs, _status = cell(inputs)

print(outputs.shape) # 모든 time-step의 hidden_state

# 마지막 층의 모든 시점의 은닉 상태
print(_status.shape) # (층의 개수, 배치 크기, 은닉 상태의 크기)

print("-"*100)

# 양방향 순환 신경망(Bidirectional Recurrent Neural Network)

# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True, bidirectional = True)

outputs, _status = cell(inputs)

print(outputs.shape) # (배치 크기, 시퀀스 길이, 은닉 상태의 크기 x 2)
# 양방향의 은닉 상태 값들이 연결(concatenate)되었기 때문

print(_status.shape) # (층의 개수 x 2, 배치 크기, 은닉 상태의 크기)
# 정방향 기준으로는 마지막 시점에 해당되며
# 역방향 기준에서는 첫번째 시점에 해당되는 시점의 출력값을 층의 개수만큼 쌓아 올린 결과값

# yt는 출력층을 사용할 때, 위의 코드는 은닉층만 표현, 위의 코드에서 yt는 고려하지 않았다.


torch.Size([1, 10, 8])
torch.Size([1, 1, 8])
----------------------------------------------------------------------------------------------------
torch.Size([1, 10, 8])
torch.Size([2, 1, 8])
----------------------------------------------------------------------------------------------------
torch.Size([1, 10, 16])
torch.Size([4, 1, 8])


# 07-03 문자 단위 RNN(Char RNN): 실습 2개

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

input_str = 'apple'
label_str = 'pple!'

# set(...): 중복된 문자를 제거하고 고유한 문자 집합을 만듬
# list(...): 집합(set)을 리스트 바꿈.
# sorted(...): 알파벳 순으로 정렬 (['!', 'a', 'e', 'l', 'p'])
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

input_size = vocab_size # 입력의 크기는 문자 집합의 크기(5개)
hidden_size = 5
output_size = 5
learning_rate = 0.1

# char_vocab는 리스트임.
# enumerate는 각 문자에 인덱스 붙여줌.
# (c, i) for i, c in enumerate(char_vocab) : enumerate 결과를 (문자, 인덱스) 순서로 바꿔서 하나씩 꺼내 표현
# dict는 키(key)와 값(value) 쌍을 저장하는 자료형
# dict(...)는 이런 쌍들을 받아서 딕셔너리를 만들어주는 함수

char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(char_to_index)

# index_to_char = {} 는 숫자(인덱스)를 문자로 되돌리기 위한 딕셔너리
# .items()	딕셔너리의 (key, value) 쌍들을 반복
# index_to_char[숫자] = 문자로 저장했기에 문자 → 숫자에서 숫자 → 문자로 바뀜
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

# char_to_index[c], c는 현재 문자, 'a'면 char_to_index['a']는 → 1 (문자를 숫자로 바꿔주는 역할)
# input_str이 'apple', ['a', 'p', 'p', 'l', 'e']로 하나씩 문자 꺼냄
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

# 배치 차원 추가
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있었음.

# RNN은 항상 입력 = [배치 수, 시퀀스 길이, 입력 벡터 차원]으로 받기에
# 한 겹 감싸서 2차원을 3차원으로 확장할 준비
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

print("-"*100)

# np.eye(...)[x]	해당 숫자 인덱스를 원-핫 벡터로 변환

# np.eye(n)은 n × n 단위 행렬(identity matrix)을 자동으로 만들어줌(주로 원-핫 벡터 만들 때 인덱싱해서 사용)
# np.array(...)는 직접 만든 리스트나 값을 넘파이 배열로 변환(리스트나 다른 구조를 묶어서 배열로 변환)
# np.eye(n)는 원-핫 벡터 만들기, 수학용이고 np.array([...])는 리스트 → 넘파이 배열로 묶는 목적

# 바깥에 [ ]가 한 번 더 있어 3차원으로 만들어 줌
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

# np.array([...])는 그 리스트를 하나의 커다란 배열로 통합(안할 경우 매우 느려지는 경고 표시 뜸!)
x_one_hot = np.array(x_one_hot)
print(x_one_hot)

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

print("-"*100)

class Net(torch.nn.Module):
  # 모델 구조 정의
  def __init__(self, input_size, hidden_size, output_size):

    # “지금 정의 중인 Net 클래스와 그 인스턴스(self)가 상속받은
    # 부모 클래스의 초기화 함수(__init__())를 실행해줘!”
    super(Net, self).__init__() # Net, self	현재 클래스(Net)와 그 인스턴스(self)
    self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
    self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

  # 데이터 흐름 정의 (입력 → RNN → FC → 출력)
  def forward(self, x): # 구현한 RNN 셀과 출력층을 연결

    # x는 전체 시퀀스에 대한 은닉 상태
    # _status는 마지막 시점의 은닉 상태 (state)
    # 입력 → RNN → 은닉 상태 x → FC(출력층) → 최종 출력 x
    # RNN이 구한 전체 은닉 상태 x를 FC층(출력층)에 넣어서 예측 결과로 변환된 x가 최종 출력
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

net = Net(input_size, hidden_size, output_size) # ← 여기서 net이 실제 self가 되는 인스턴스 이름

outputs = net(X)
print(outputs.shape) # 3차원 텐서

# -1 : "전체 데이터 수는 유지하고, 두 번째 차원은 5로 하고, 나머지는 알아서 계산해줘!"

# 총 원소 수 = 1 × 5 × 5 = 25를 .view(-1, 5)하면 두 번째 차원을 5로 지정된 상태에
# 전체 원소 수는 그대로 25개여야 하며 -1로 자동 계산해달라 했으니
# 첫 번째 차원은 자동으로 계산되어 5가 나옴(첫 번째 차원 = 25 / 5 = 5)

print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환

# Y의 총 원소는 1 × 5 = 5이고 -1로 전체 원소 수는 유지하며 1차원으로 자동 계산하니 5가 나옴.
print(Y.shape)
print(Y.view(-1).shape)

criterion = torch.nn.CrossEntropyLoss() # 손실 함수를 정의
optimizer = optim.Adam(net.parameters(), learning_rate) # 옵티마이저(기울기) 정의

print("-"*100)

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    # # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택

    # axis=2 : 클래스 차원 각 시점에서 가장 높은 클래스 인덱스 추출
    # 각 행(=각 시점의 출력)에서 가장 높은 값의 인덱스(번호)를 추출
    result = outputs.data.numpy().argmax(axis=2)

    # np.squeeze()는 "차원을 줄이는 함수"
    # np.squeeze(result): [1, 5] 형태의 배열에서 불필요한 배치 차원(1) 제거 → [5]

    # [index_to_char[c] for c in ...]: 각 인덱스를 문자로 복원 (예: 2 → 'p', 4 → 'l' 등

    # join()은 리스트 요소들을 하나의 문자열로 합치는 함수
    # ''는 각 요소 사이에 아무것도 넣지 말라는 뜻
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)



문자 집합의 크기 : 5
{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}
{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}
[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]
[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]
----------------------------------------------------------------------------------------------------
[[[0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 1. 0.]
  [0. 0. 1. 0. 0.]]]
훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])
----------------------------------------------------------------------------------------------------
torch.Size([1, 5, 5])
torch.Size([5, 5])
torch.Size([1, 5])
torch.Size([5])
----------------------------------------------------------------------------------------------------
0 loss:  1.5977658033370972 prediction:  [[0 0 0 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  !!!!!
1 loss:  1.3395131826400757 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
2 loss:  1.1239253282546997 prediction:  [[4 4 4 2 4]] true Y:  [[4, 4, 3, 2, 0]] pred

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩
print(char_dic) # 공백도 여기서는 하나의 원소

dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

print("-"*100)

# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10  # 임의 숫자 지정
learning_rate = 0.01

# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):

  # 슬라이싱 때문에 "- sequence_length" 한 것(160 - 10 = 150, range(0, 150)이면 0~149개가 되며
  # i = 149일 때 i + 149 = 159, i + 149 + 1 = 160으로 단어가 안 깨짐.)

  # 입력과 출력이 한 글자 밀린 이유는 → "이전 문자들을 보고 다음 문자를 예측"하는 구조
  x_str = sentence[i:i + sequence_length]
  y_str = sentence[i + 1: i + sequence_length + 1]
  print(i, x_str, '->', y_str)

  x_data.append([char_dic[c] for c in x_str])  # x str to index
  y_data.append([char_dic[c] for c in y_str])  # y str to index

print("-"*100)

print(x_data[0]) # if you wan에 해당됨.
print(y_data[0]) # f you want에 해당됨.

print("-"*100)

x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
x_one_hot = np.array(x_one_hot)

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

print(X[0])

print("-"*100)

print(Y[0])

print("-"*100)

class Net(torch.nn.Module):
  # num_layers=layers는 몇 층(layer)의 RNN을 쌓을 것인지를 설정하는 하이퍼파라미터
  def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
      super(Net, self).__init__()
      self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
      self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

  def forward(self, x):
      x, _status = self.rnn(x)
      x = self.fc(x)
      return x

net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

outputs = net(X)
print(outputs.shape) # 3차원 텐서

print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환.(dic_size = 25)

print("-"*100)

# 레이블 데이터는 (170, 10)의 크기를 가짐, 나중에 정확도를 측정할 때는 이걸 펼쳐서 계산할 예정
print(Y.shape)
print(Y.view(-1).shape)

print("-"*100)

for i in range(175):
  optimizer.zero_grad()
  outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
  loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
  loss.backward()
  optimizer.step()

  # results의 텐서 크기는 (170, 10)
  results = outputs.argmax(dim=2)
  predict_str = "" # ""는 빈 문자열(empty string) 의미

  # enumerate는 리스트나 문자열을 반복할 때, 각 요소에 번호(index)를 자동으로 붙여주는 함수
  for j, result in enumerate(results):
    input_sentence = ''.join([char_set[i] for i in x_data[0]])
    if j == 0: #  첫 번째 시점에는 예측된 전체 시퀀스 결과를 전부 가져오지만
      # char_set[t]를 통해 다시 문자로 변환
      # ''.join(...)는 문자들을 하나의 문자열로 합침
      predict_str += ''.join([char_set[t] for t in result])
    else: # 그 다음에는 마지막 글자만 하나씩 하나씩 이어붙이는 방식
      # result[-1]은 그 리스트의 마지막 원소
      # 시점 j=1 이후부터는 문장을 한 글자씩 예측하니
      # 예측한 마지막 글자 하나만 꺼내서 최종 출력 문자열에 덧붙이는 역할
      predict_str += char_set[result[-1]]

  predict_str = input_sentence[0] + predict_str  # 첫 글자 'i'를 앞에 붙임
  print(predict_str)


# RNN은 “입력의 첫 글자를 보고 → 그 다음 글자를 예측
# 그다음 글자까지 보고 → 그 다음 글자 또 예측, 이런 식으로 하나씩 다음 시점을 예측해가는 구조

# 입력 시퀀스 "if you wan" → 출력 시퀀스 "f you want"
# 'i'는 모델이 직접 예측한 적은 없음 (입력으로만 주어짐), 훈련도 "f"부터 예측하는 식으로 되어 있음
# 훈련 중에 'i'를 출력하는 학습이 안 됨,
# 그래서 테스트 시점에도 'i'를 출력하는 방법을 모르고, 임의의 글자 'g'를 뽑아냄

# input_sentence = ''.join([char_set[i] for i in x_data[0]])을 함으로써 input_sentence를 정의함!
# predict_str = input_sentence[0] + predict_str 이걸 함으로써 'i'를 첫 글자로 넣음(예측 가능해짐!)
# 처음에는 이상한 예측을 하지만 마지막 에포크에서는 꽤 정확한 문자을 생성했음.(학습률 0.01, epoch 175)


{'u': 0, 'b': 1, 't': 2, 'a': 3, 'd': 4, 'c': 5, '.': 6, ',': 7, 'k': 8, 'y': 9, 'h': 10, 's': 11, 'r': 12, 'o': 13, ' ': 14, 'f': 15, 'i': 16, 'e': 17, 'g': 18, 'p': 19, 'l': 20, 'n': 21, "'": 22, 'm': 23, 'w': 24}
문자 집합의 크기 : 25
----------------------------------------------------------------------------------------------------
0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip